In [1]:
import pandas as pd
import numpy as np
import scipy
import nltk
import spacy
import gensim
import glob
import csv
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import cross_val_score
import sklearn.model_selection
import sklearn.pipeline
import re
from sklearn import svm
from sklearn import *
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.feature_selection import chi2
from sklearn.base import BaseEstimator, TransformerMixin
import gensim.models.wrappers.fasttext
from scipy import sparse
import tensorflow_datasets as tfds
import tensorflow as tf
import collections
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import LeaveOneOut,KFold,train_test_split
import pprint
from tensorflow.keras.layers import Layer, Dense, Flatten, Activation, Permute
from tensorflow.keras.layers import Multiply, Lambda, Reshape, Dot, Concatenate, RepeatVector, \
    TimeDistributed, Permute, Bidirectional



from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers


# Custom imports
from mr_generic_scripts import *
from mr_cls_Attention import *

In [2]:
# Path to xlsx files folder
path_to_raw_files = 'Data/raw/'


In [3]:
# Get the datasets in dataframes
a_dataset = mr_get_data(path_to_raw_files)
qa_dataset = mr_get_qa_data(path_to_raw_files)

In [4]:
# Sanity check
print(a_dataset[0][1][:5])
print(qa_dataset[0][1][:5])
print(len(a_dataset[0][1]))

   Child_ID                                             Answer  Score  Age  \
0         0   Because they did n't want the woman to find them      2    9   
1         1           Because they Were n't supose to be there      0    9   
2         2                                they have a seccret      0    9   
3         3               because they didnt whant to get cort      1    9   
4         4  Because they did n't want the woman to see them .      1    9   

   Gender           Question  
0       0  SFQuestion_1_Text  
1       1  SFQuestion_1_Text  
2       0  SFQuestion_1_Text  
3       0  SFQuestion_1_Text  
4       1  SFQuestion_1_Text  
   Child_ID                                             Answer  Score  Age  \
0         0  Why did the men hide ? Because they did n't wa...      2    9   
1         1  Why did the men hide ? Because they Were n't s...      0    9   
2         2         Why did the men hide ? they have a seccret      0    9   
3         3  Why did the men hide

In [5]:
# Create a new instalce of the classifier
# We use the predefined column names, age list from 7 to 14, vocabulary size of 20, padding/trunkating of 20
att_cls = MR_attention(text_cols,[7,8,9,10,11,12,13,14],1000,20)

In [6]:
# Set evaluation parameters - evaluate by question, evaluate by age; do not return incorrect predictions (experimental)
att_cls.mr_set_eval_vars(True,True,False)

In [7]:
# 10 fold cross validation, original dataset, without the question
# 25% split for validation (each run is 67.5 / 22.5 / 10 for train / val / test)
results_1 = att_cls.mr_kfold_train_test(a_dataset[-1][1],0.25,10)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 64)            64000     
_________________________________________________________________
self_attention (SelfAttentio (None, 384)               1400      
_________________________________________________________________
flatten (Flatten)            (None, 384)               0         
_________________________________________________________________
dense (Dense)                (No

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 64)            64000     
_________________________________________________________________
self_attention_1 (SelfAttent (None, 384)               1400      
_________________________________________________________________
flatten_1 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                7700      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
__________________________________________

1909/1909 [==============================] - 6s 3ms/step - loss: 1.0857 - accuracy: 0.4487 - val_loss: 1.0306 - val_accuracy: 0.6500
Epoch 2/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.9468 - accuracy: 0.6441 - val_loss: 0.8665 - val_accuracy: 0.7250
Epoch 3/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.8376 - accuracy: 0.7404 - val_loss: 0.7919 - val_accuracy: 0.7500
Epoch 4/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7867 - accuracy: 0.7834 - val_loss: 0.7480 - val_accuracy: 0.8083
Epoch 5/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7577 - accuracy: 0.8102 - val_loss: 0.7143 - val_accuracy: 0.8333
Epoch 6/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7359 - accuracy: 0.8295 - val_loss: 0.7366 - val_accuracy: 0.8000
Epoch 7/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7240 - accuracy: 0.8415 - val_loss: 0.7205 - val_accuracy: 0.8333
Epo

1909/1909 [==============================] - 6s 3ms/step - loss: 0.8322 - accuracy: 0.7497 - val_loss: 0.7924 - val_accuracy: 0.7667
Epoch 4/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7861 - accuracy: 0.7935 - val_loss: 0.7301 - val_accuracy: 0.8417
Epoch 5/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7586 - accuracy: 0.8166 - val_loss: 0.7813 - val_accuracy: 0.7750
Epoch 6/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7438 - accuracy: 0.8242 - val_loss: 0.7476 - val_accuracy: 0.8083
Epoch 7/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7272 - accuracy: 0.8420 - val_loss: 0.7809 - val_accuracy: 0.7583
Epoch 8/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7170 - accuracy: 0.8502 - val_loss: 0.7437 - val_accuracy: 0.8083
Epoch 9/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7114 - accuracy: 0.8536 - val_loss: 0.7314 - val_accuracy: 0.8083
Epo

1909/1909 [==============================] - 6s 3ms/step - loss: 0.7513 - accuracy: 0.8174 - val_loss: 0.7581 - val_accuracy: 0.7833
Epoch 6/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7354 - accuracy: 0.8292 - val_loss: 0.7421 - val_accuracy: 0.8167
Epoch 7/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7267 - accuracy: 0.8334 - val_loss: 0.7301 - val_accuracy: 0.8083
Epoch 8/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7143 - accuracy: 0.8478 - val_loss: 0.7210 - val_accuracy: 0.8417
Epoch 9/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7078 - accuracy: 0.8517 - val_loss: 0.7276 - val_accuracy: 0.8417
Epoch 10/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7024 - accuracy: 0.8557 - val_loss: 0.7168 - val_accuracy: 0.8500
Epoch 11/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.6965 - accuracy: 0.8618 - val_loss: 0.7048 - val_accuracy: 0.8417
E

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 64)            64000     
_________________________________________________________________
self_attention_5 (SelfAttent (None, 384)               1400      
_________________________________________________________________
flatten_5 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_10 (Dense)             (

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 64)            64000     
_________________________________________________________________
self_attention_6 (SelfAttent (None, 384)               1400      
_________________________________________________________________
flatten_6 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 20)                7700      
_________________________________________________________________
dropout_6 (Dropout)          (None, 20)                0         
__________________________________________

1909/1909 [==============================] - 6s 3ms/step - loss: 1.0787 - accuracy: 0.4406 - val_loss: 1.0337 - val_accuracy: 0.4917
Epoch 2/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.9547 - accuracy: 0.6096 - val_loss: 0.9314 - val_accuracy: 0.6583
Epoch 3/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.8545 - accuracy: 0.7232 - val_loss: 0.8176 - val_accuracy: 0.7500
Epoch 4/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7875 - accuracy: 0.7796 - val_loss: 0.7856 - val_accuracy: 0.7750
Epoch 5/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7542 - accuracy: 0.8127 - val_loss: 0.7743 - val_accuracy: 0.7667
Epoch 6/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7380 - accuracy: 0.8227 - val_loss: 0.7331 - val_accuracy: 0.8250
Epoch 7/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7261 - accuracy: 0.8324 - val_loss: 0.7155 - val_accuracy: 0.8417
Epo

1909/1909 [==============================] - 6s 3ms/step - loss: 0.8905 - accuracy: 0.6769 - val_loss: 0.8433 - val_accuracy: 0.6917
Epoch 4/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.8253 - accuracy: 0.7439 - val_loss: 0.8283 - val_accuracy: 0.7083
Epoch 5/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7814 - accuracy: 0.7856 - val_loss: 0.8225 - val_accuracy: 0.6917
Epoch 6/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7590 - accuracy: 0.8039 - val_loss: 0.7706 - val_accuracy: 0.7583
Epoch 7/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7407 - accuracy: 0.8224 - val_loss: 0.7851 - val_accuracy: 0.7583
Epoch 8/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7280 - accuracy: 0.8342 - val_loss: 0.7868 - val_accuracy: 0.7583
Epoch 9/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7214 - accuracy: 0.8371 - val_loss: 0.7739 - val_accuracy: 0.7583
Epo

1909/1909 [==============================] - 6s 3ms/step - loss: 0.7683 - accuracy: 0.7984 - val_loss: 0.7364 - val_accuracy: 0.8250
Epoch 6/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7454 - accuracy: 0.8161 - val_loss: 0.7334 - val_accuracy: 0.8167
Epoch 7/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7302 - accuracy: 0.8309 - val_loss: 0.6986 - val_accuracy: 0.8417
Epoch 8/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7196 - accuracy: 0.8396 - val_loss: 0.6643 - val_accuracy: 0.8917
Epoch 9/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7115 - accuracy: 0.8485 - val_loss: 0.6736 - val_accuracy: 0.8750
Epoch 10/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7059 - accuracy: 0.8525 - val_loss: 0.7107 - val_accuracy: 0.8500
Epoch 11/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7012 - accuracy: 0.8558 - val_loss: 0.7103 - val_accuracy: 0.8417
E

In [8]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(results_1)

[   (   [   0.81,
            [0.81, 0.84, 0.84, 0.81, 0.83, 0.72, 0.81, 0.8, 0.81, 0.86, 0.78],
            [0.89, 0.71, 0.83, 0.78, 0.83, 0.81, 0.86, 1.0]],
        [   0.81,
            [0.64, 0.75, 0.76, 0.77, 0.81, 0.7, 0.65, 0.56, 0.78, 0.75, 0.73],
            [0.9, 0.7, 0.83, 0.78, 0.83, 0.81, 0.85, 1.0]]),
    (   [   0.81,
            [0.78, 0.72, 0.83, 0.82, 0.84, 0.81, 0.92, 0.78, 0.79, 0.83, 0.78],
            [0.89, 0.8, 0.82, 0.81, 0.84, 0.79, 0.73, 1.0]],
        [   0.81,
            [0.6, 0.61, 0.75, 0.78, 0.8, 0.78, 0.8, 0.54, 0.78, 0.79, 0.76],
            [0.86, 0.81, 0.82, 0.81, 0.83, 0.77, 0.7, 1.0]]),
    (   [   0.8,
            [0.79, 0.75, 0.85, 0.8, 0.79, 0.76, 0.85, 0.85, 0.79, 0.88, 0.67],
            [0.83, 0.78, 0.81, 0.78, 0.79, 0.8, 0.8, 1.0]],
        [   0.79,
            [0.59, 0.64, 0.78, 0.8, 0.76, 0.69, 0.57, 0.77, 0.76, 0.8, 0.62],
            [0.84, 0.78, 0.82, 0.78, 0.79, 0.79, 0.79, 1.0]]),
    (   [   0.8,
            [0.78, 0.75, 0.82, 0.74

In [7]:
def mr_proc_results(raw_results):
    # Process the results from the 10 runs
    # result format: [acc, acc per q, acc per age], [f1, f1 per q, f1 per age]
    # Ignore ages as they seem to be mostly consistent with global average
    # Ignore accs per question and age as averaging them seems to be consistent with global average
    # Report global acc, global macro f1, average of macro f1 per question
    pr_results = [[acc_score, f1_score,round(sum(qa_s)/11,2),round(sum(qf_s)/11,2)] 
                for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s]) in raw_results]
    
    # Throw the list in an np array
    pr_arr = np.array(pr_results)

    # Print the results
    pp = pprint.PrettyPrinter(indent=4)

    print("Acc, F1, Avg-F1-Per-Q")
    pp.pprint(pr_results)
    pp.pprint(np.mean(pr_arr,axis=0))
    
    
    print("\nAcc per question (avg over 10 runs)")
    # Get the result per question
    pr_qa_results = [[qa_s] for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s]) in raw_results]
    pr_qa_arr = np.array(pr_qa_results)
    avg_qa = np.mean(pr_qa_arr,axis=0).tolist()
    print(' & '.join([str(round(res,2)) for res in avg_qa[0]]))
    #pp.pprint(np.mean(pr_qa_arr,axis=0))
    
    print("\nF1 per question (avg over 10 runs)")
    pr_qf_results = [[qf_s] for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s]) in raw_results]
    pr_qf_arr = np.array(pr_qf_results)
    avg_qf = np.mean(pr_qf_arr,axis=0).tolist()
    print(' & '.join([str(round(res,2)) for res in avg_qf[0]]))
    #pp.pprint(np.mean(pr_qf_arr,axis=0))

In [10]:
mr_proc_results(results_1)

Acc, F1, Avg-F1-Per-Q
[   [0.81, 0.81, 0.81, 0.72],
    [0.81, 0.81, 0.81, 0.73],
    [0.8, 0.79, 0.8, 0.71],
    [0.8, 0.8, 0.8, 0.71],
    [0.81, 0.8, 0.81, 0.69],
    [0.81, 0.81, 0.81, 0.72],
    [0.82, 0.81, 0.81, 0.74],
    [0.81, 0.81, 0.81, 0.73],
    [0.83, 0.83, 0.83, 0.76],
    [0.8, 0.8, 0.8, 0.72]]
array([0.81 , 0.807, 0.809, 0.723])

Acc per question (avg over 10 runs)
0.8 & 0.78 & 0.84 & 0.8 & 0.83 & 0.76 & 0.85 & 0.85 & 0.79 & 0.85 & 0.75

F1 per question (avg over 10 runs)
0.65 & 0.69 & 0.77 & 0.77 & 0.79 & 0.72 & 0.67 & 0.68 & 0.75 & 0.77 & 0.71


In [8]:
# Change the object parameters - including the question increasees the max length to 35
att_cls.mr_set_model_vars(text_cols,[7,8,9,10,11,12,13,14],1000,35)

In [9]:
# 10 fold cross validation, original dataset, including the question
# 25% split for validation (each run is 67.5 / 22.5 / 10 for train / val / test)
results_2 = att_cls.mr_kfold_train_test(qa_dataset[-1][1],0.25,10)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 35, 64)            64000     
_________________________________________________________________
self_attention (SelfAttentio (None, 384)               2450      
_________________________________________________________________
flatten (Flatten)            (None, 384)               0         
_________________________________________________________________
dense (Dense)                (No

C:\Users\Venelin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


73/73 [==============================] - 0s 2ms/step - loss: 0.7277 - accuracy: 0.8241
Macro F1: 0.82 

Evaluating column Age with value 10
78/78 [==============================] - 0s 2ms/step - loss: 0.7500 - accuracy: 0.7961
Macro F1: 0.8 

Evaluating column Age with value 11
57/57 [==============================] - 0s 2ms/step - loss: 0.7065 - accuracy: 0.8421
Macro F1: 0.84 

Evaluating column Age with value 12
25/25 [==============================] - 0s 2ms/step - loss: 0.6562 - accuracy: 0.8969
Macro F1: 0.9 

Evaluating column Age with value 13
32/32 [==============================] - 0s 2ms/step - loss: 0.7025 - accuracy: 0.8516
Macro F1: 0.84 

Evaluating column Age with value 14
1/1 [==============================] - 0s 5ms/step - loss: 0.5514 - accuracy: 1.0000
Macro F1: 1.0 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'c

54/54 [==============================] - 0s 2ms/step - loss: 0.7063 - accuracy: 0.8404
Macro F1: 0.83 

Evaluating column Age with value 12
37/37 [==============================] - 0s 2ms/step - loss: 0.7159 - accuracy: 0.8299
Macro F1: 0.81 

Evaluating column Age with value 13
27/27 [==============================] - 0s 2ms/step - loss: 0.7700 - accuracy: 0.7778
Macro F1: 0.75 

Evaluating column Age with value 14
1/1 [==============================] - 0s 4ms/step - loss: 0.5515 - accuracy: 1.0000
Macro F1: 1.0 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Model: "sequential_2"
_____________________________________

30/30 [==============================] - 0s 2ms/step - loss: 0.7330 - accuracy: 0.8051
Macro F1: 0.8 

Evaluating column Age with value 14
1/1 [==============================] - 0s 5ms/step - loss: 0.5530 - accuracy: 1.0000
Macro F1: 1.0 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 35, 64)            64000     
_________________________________________________________________
self_attention_3 (SelfAttent (None, 384)              

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 35, 64)            64000     
_________________________________________________________________
self_attention_5 (SelfAttent (None, 384)               2450      
_________________________________________________________________
flatten_5 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_10 (Dense)             (

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'astor' has no attribute 'code_gen'
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 35, 64)            64000     
_________________________________________________________________
self_attention_6 (SelfAttent (None, 384)               2450      
_________________________________________________________________
flatten_6 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 35)                13475     
_________________________________________________________________
dropout_6 (Dropout)          (None, 35)                0         
__________________________________________

1909/1909 [==============================] - 7s 4ms/step - loss: 1.0701 - accuracy: 0.4861 - val_loss: 0.9940 - val_accuracy: 0.6667
Epoch 2/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.8605 - accuracy: 0.7305 - val_loss: 0.7957 - val_accuracy: 0.7667
Epoch 3/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7655 - accuracy: 0.8030 - val_loss: 0.7524 - val_accuracy: 0.7833
Epoch 4/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7320 - accuracy: 0.8303 - val_loss: 0.7509 - val_accuracy: 0.8083
Epoch 5/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7156 - accuracy: 0.8423 - val_loss: 0.7417 - val_accuracy: 0.8167
Epoch 6/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7034 - accuracy: 0.8538 - val_loss: 0.7784 - val_accuracy: 0.7917
Epoch 7/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.6966 - accuracy: 0.8585 - val_loss: 0.7715 - val_accuracy: 0.7833
Epo

1909/1909 [==============================] - 6s 3ms/step - loss: 0.7673 - accuracy: 0.8021 - val_loss: 0.7443 - val_accuracy: 0.8083
Epoch 4/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7353 - accuracy: 0.8274 - val_loss: 0.7437 - val_accuracy: 0.8000
Epoch 5/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7155 - accuracy: 0.8440 - val_loss: 0.7370 - val_accuracy: 0.7917
Epoch 6/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7042 - accuracy: 0.8530 - val_loss: 0.7444 - val_accuracy: 0.8000
Epoch 7/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.6956 - accuracy: 0.8588 - val_loss: 0.7612 - val_accuracy: 0.7833
Epoch 8/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.6908 - accuracy: 0.8623 - val_loss: 0.7379 - val_accuracy: 0.8000
Epoch 9/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.6828 - accuracy: 0.8735 - val_loss: 0.7892 - val_accuracy: 0.7500
Epo

1909/1909 [==============================] - 6s 3ms/step - loss: 0.7148 - accuracy: 0.8437 - val_loss: 0.6722 - val_accuracy: 0.8917
Epoch 6/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.7043 - accuracy: 0.8533 - val_loss: 0.7167 - val_accuracy: 0.8250
Epoch 7/20
1909/1909 [==============================] - 7s 4ms/step - loss: 0.6959 - accuracy: 0.8605 - val_loss: 0.6926 - val_accuracy: 0.8583
Epoch 8/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.6868 - accuracy: 0.8690 - val_loss: 0.7033 - val_accuracy: 0.8583
Epoch 9/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.6834 - accuracy: 0.8723 - val_loss: 0.7061 - val_accuracy: 0.8500
Epoch 10/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.6771 - accuracy: 0.8787 - val_loss: 0.6924 - val_accuracy: 0.8667
Epoch 11/20
1909/1909 [==============================] - 6s 3ms/step - loss: 0.6739 - accuracy: 0.8808 - val_loss: 0.6815 - val_accuracy: 0.8667
E

In [11]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(results_2)

[   (   [   0.83,
            [0.81, 0.81, 0.87, 0.79, 0.83, 0.77, 0.84, 0.84, 0.76, 0.82, 0.92],
            [0.78, 0.75, 0.82, 0.8, 0.84, 0.9, 0.85, 1.0]],
        [   0.82,
            [0.59, 0.7, 0.83, 0.73, 0.8, 0.75, 0.62, 0.58, 0.72, 0.73, 0.89],
            [0.54, 0.75, 0.82, 0.8, 0.84, 0.9, 0.84, 1.0]]),
    (   [   0.83,
            [0.8, 0.74, 0.84, 0.82, 0.85, 0.8, 0.88, 0.85, 0.84, 0.88, 0.85],
            [1.0, 0.82, 0.82, 0.85, 0.84, 0.83, 0.78, 1.0]],
        [   0.83,
            [0.58, 0.65, 0.77, 0.79, 0.81, 0.76, 0.64, 0.58, 0.83, 0.8, 0.85],
            [1.0, 0.82, 0.82, 0.84, 0.83, 0.81, 0.75, 1.0]]),
    (   [   0.82,
            [0.84, 0.71, 0.84, 0.81, 0.81, 0.75, 0.85, 0.9, 0.81, 0.9, 0.78],
            [0.83, 0.85, 0.83, 0.82, 0.82, 0.78, 0.81, 1.0]],
        [   0.82,
            [0.62, 0.61, 0.76, 0.76, 0.77, 0.68, 0.54, 0.78, 0.8, 0.81, 0.74],
            [0.78, 0.85, 0.83, 0.82, 0.81, 0.77, 0.8, 1.0]]),
    (   [   0.83,
            [0.76, 0.8, 0.86, 0.82

In [12]:
mr_proc_results(results_2)

Acc, F1, Avg-F1-Per-Q
[   [0.83, 0.82, 0.82, 0.72],
    [0.83, 0.83, 0.83, 0.73],
    [0.82, 0.82, 0.82, 0.72],
    [0.83, 0.83, 0.83, 0.73],
    [0.82, 0.82, 0.82, 0.72],
    [0.84, 0.84, 0.84, 0.75],
    [0.84, 0.84, 0.84, 0.75],
    [0.83, 0.83, 0.83, 0.73],
    [0.84, 0.84, 0.84, 0.74],
    [0.82, 0.82, 0.82, 0.72]]
array([0.82999998, 0.829     , 0.829     , 0.731     ])

Acc per question (avg over 10 runs)
0.81 & 0.76 & 0.86 & 0.81 & 0.85 & 0.77 & 0.86 & 0.89 & 0.8 & 0.86 & 0.86

F1 per question (avg over 10 runs)
0.62 & 0.66 & 0.8 & 0.74 & 0.81 & 0.73 & 0.63 & 0.73 & 0.76 & 0.74 & 0.82
